In this notebook we perform l1-regularization with different regularization strengths and compare it.
L1-regularization assumes a laplacian prior on the parameter with location $\mu=0$ and scale $b=1/\lambda$, where $\lambda$ is called the penalization strenght. objective function takes then the form
$$
J_{l_1}(\theta, \lambda)=-\log(\mathcal{p}(\mathcal{D}|\theta))+\lambda\sum_{i=1}^{n_{\theta}}|\theta_i|,
$$
where the first part is the negative log-likelihood (the normal objective function) and the second part the penality term. 
We perform the regularization in a way that we use the optimal parameters from the optimization of the regularized objective function with strength $\lambda_{k-1}$ as starting points for the optimzation with strength $\lambda_k$.